In [ ]:
# 导入函数库
from jqdata import *
from collections import namedtuple
import numpy as np

position_info = namedtuple('position_info', 'buyin count jumpflag')

# 初始化函数，设定基准等等
def initialize(context):
    #设定沪深300作为基准
    set_benchmark('000300.XSHG')
    # 开启动态复权模式(真实价格)
    set_option('use_real_price', True)
    # 输出内容到日志 log.info()
    log.info('初始函数开始运行且全局只运行一次')
    # 过滤掉order系列API产生的比error级别低的log
    # log.set_level('order', 'error')
    
    ### 股票相关设定 ###
    # 股票类每笔交易时的手续费是：买入时佣金万分之三，卖出时佣金万分之三加千分之一印花税, 每笔交易佣金最低扣5块钱
    set_order_cost(OrderCost(close_tax=0.001, open_commission=0.00025, close_commission=0.00025, min_commission=5), type='stock')

    g.security = '002594.XSHE'
    g.n_value = 0
    g.max_position_count = 12
    g.position = {}
    g.stock_start_date = get_security_info(g.security).start_date
    g.last_buyin = 0
    g.jump_flag = 0
    g.buyin_order_not_finished = []
    g.stop_price = 0
    g.last_open = 0
    
    
    # 运行函数（reference_security为运行时间的参考标的；传入的标的只做种类区分，因此传入'000300.XSHG'或'510300.XSHG'是一样的）
      # 开盘前运行
    run_daily(before_market_open, time='before_open', reference_security='000300.XSHG')
      # 开盘时运行
    run_daily(market_open, time='open', reference_security='000300.XSHG')
      # 收盘后运行
    run_daily(after_market_close, time='after_close', reference_security='000300.XSHG')


#5日均线战法
def five_days_mean(context):
    security = g.security
    # 获取股票的收盘价
    close_data = get_bars(security, count=5, unit='1d', fields=['close'])
    # 取得过去五天的平均价格
    MA5 = close_data['close'].mean()
    # 取得上一时间点价格
    current_price = close_data['close'][-1]
    # 取得当前的现金
    cash = context.portfolio.available_cash

    # 如果上一时间点价格高出五天平均价1%, 则全仓买入
    if current_price > 1.01*MA5:
        # 记录这次买入
        log.info("价格高于均价 1%%, 买入 %s" % (security))
        # 用所有 cash 买入股票
        order_value(security, cash)
    # 如果上一时间点价格低于五天平均价, 则空仓卖出
    elif current_price < MA5 and context.portfolio.positions[security].closeable_amount > 0:
        # 记录这次卖出
        log.info("价格低于均价, 卖出 %s" % (security))
        # 卖出所有股票,使这只股票的最终持有量为0
        order_target(security, 0)
#####################################################################################


def get_atr(data):
    return max(data[0]-data[1], abs(data[0]-data[2]), abs(data[2]-data[1]))


def get_n_initial_value(context):
    stock_data = get_price(g.security, end_date=context.current_dt.date(), frequency='daily', fields=['high', 'low', 'pre_close'], count=20)
    if len(stock_data.dropna(how='any')) < 20:
        log.info("data is missing, next")
        return 0
    return np.array([get_atr(x) for x in stock_data.values]).mean()

    
def get_n_value(context):
    stock_data = get_price(g.security, end_date=context.current_dt.date(), frequency='daily', fields=['high', 'low', 'pre_close'], count=1)
    # log.info(stock_data.values.tolist()[0])
    return (19*g.n_value + get_atr(stock_data.values.tolist()[0]))/20


def process_n_value(context):
    if (0 == g.n_value):
        g.n_value = get_n_initial_value(context)
        # log.info('n initial value is %s' % g.n_value)
    else:
        g.n_value = get_n_value(context)
        # log.info('n value is %s' % g.n_value)

    
def calculate_market_in_and_out_price_with_long(context, bt_in_period, bt_out_period):
    stock_data = get_price(g.security, end_date=context.current_dt.date(), frequency='daily', fields=['high', 'low'], count=bt_in_period)
    # log.info(stock_data)
    return {"in_val": stock_data[:]['high'].max(), "out_val": stock_data[bt_out_period:]['low'].min()}


def buyin_with_method_1(pu, buyin_value, position_count, half_n_value, double_n_value, high_value):
    stop_price = 0
    tmp_position = []
    left_position_count = g.max_position_count - position_count
    for i in range(0,left_position_count):
        buyin_value = buyin_value + i*half_n_value
        if (buyin_value > high_value):
            log.info('buyin value :%s' % buyin_value, 'over current high value: %s' % high_value)
            break;
        # log.info('left_position_count: %s' % left_position_count, 'buy in value %s' % buyin_value, 'half_n_value: %s' % half_n_value)
        oo = order_value(g.security, pu, LimitOrderStyle(buyin_value))
        if oo is not None:
            # log.info('buy position unit:%s'% pu, 'as want price: %s' % buyin_value, 'actual price :%s'%oo.price, \
            # 'order amount:%s'%oo.amount, 'filled conunt:%s'%oo.filled, 'order status:%s'%oo.status, 'succeesfully')
            if oo.status != OrderStatus.held:
                # log.info('order id:%s'%oo.order_id,'is not finished, status is :%s'%oo.status,'continue...')
                g.buyin_order_not_finished.append(oo.order_id)
                continue
            stop_price = oo.price-double_n_value
            # log.info('method 1 stop price %s'%stop_price, 'jumpflag: %s'%g.jump_flag)
            tmp_position.append(position_info(buyin=oo.price, count=oo.filled, jumpflag=g.jump_flag))
            g.last_buyin = oo.price
        else:
            log.info("buy in failed, continue")
    if stop_price == 0:
        log.info("no buy in action, return")
        return
    
    if position_count != 0:
        max_exsit_stop_price = max(list(g.position.keys()))
        max_exsit_stop_price_value = g.position[max_exsit_stop_price]
        if max_exsit_stop_price_value[0].jumpflag == g.jump_flag:
            tmp_position += max_exsit_stop_price_value
            del g.position[max_exsit_stop_price]
            # log.info("combine last stop price:%s"%max_exsit_stop_price,"position elements")
            
    g.position[stop_price] = tmp_position
    g.stop_price = stop_price
    # for key in list(g.position.keys()):
    #     for pi in g.position[key]:
    #         log.info('now position info, stop price:%s'%key, 'buyin :%s'%pi.buyin, 'count:%s'%pi.count, 'jumpflag: %s'%pi.jumpflag)
  
  
def buyin_with_method_2():
    log.info("method 2 is not supported now")
    

def buy_in_market(context, bt_data, open_value, high_value, method=1):
    log.info('ready to buy in %s...'%g.security)
    position_count=0
    for x in g.position.values():
        position_count += len(x)
    # log.info('position count:%s'%position_count)
    if position_count >= g.max_position_count:
        # log.info('position count reach to max 12, no in market deal going')
        return
    
    pu = context.portfolio.available_cash/(g.max_position_count-position_count)
    # log.info('position unit is %s' % pu)

    half_n_value = g.n_value/2
    double_n_value = 2*g.n_value
    buyin_value = 0
    
    buyin_value = g.last_buyin + half_n_value
    if position_count == 0:
        buyin_value = g.last_buyin = bt_data['in_val']
    if open_value - g.last_buyin >= half_n_value:
        # log.info('open price:%s'%open_value, 'is higher than last buy in price:%s'%g.last_buyin, '+ half n value :%s'%half_n_value, 'make last price equal with open price')
        buyin_value = open_value
        g.jump_flag+=1

    if method == 1:
        buyin_with_method_1(pu, buyin_value, position_count, half_n_value, double_n_value, high_value)
    else:
        buyin_with_method_2()
    log.info('buy in finished...')  
    
    
def sale_out_market(context, bt_data, low_value, method=1):
    log.info("ready to sale out %s..." % g.security)
    position_count=0
    for x in g.position.values():
        position_count += len(x)
    # log.info('position count:%s'%position_count)
    if position_count <= 0:
        log.info('position count reach to minmum, no out market deal going')
        return
    
    # log.info('low_value:%s'%low_value, 'out_value:%s'%bt_data['out_val'])
    is_sale_out_success = True
    if bt_data['out_val'] >= low_value:
        # log.info('sale out all position unit, sale out breakthrough value:%s'%bt_data['out_val'], 'current low value:%s'%low_value)
        oto = order_target_value(g.security, 0, LimitOrderStyle(bt_data['out_val']))
        if oto is not None:
            # log.info('sale out all position unit as price :%s'%oto.price, 'want price:%s'%bt_data['out_val'],'successfully' )
            g.position.clear()
            g.jump_flag = 0
    else:
        ready_for_delete = {}
        for x in list(g.position.keys()):
            ready_for_delete[x] = False
        log.info(ready_for_delete)
        for stop_price in list(g.position.keys()):
            if low_value <= stop_price:
                for position_info in g.position[stop_price]:
                    # log.info('stop price:%s'%stop_price, 'sale out count:%s'%position_info.count)
                    oo = order(g.security, -position_info.count, LimitOrderStyle(stop_price))
                    if oo is not None:
                        log.info('sale out order amount:%s'%oo.amount, 'filled conunt:%s'%oo.filled, 'want count:%s'%position_info.count,'as price %s'%oo.price, 'want price:%s'%stop_price, 'successfully')
                    else:
                        log.info('sale out failed, break')
                        is_sale_out_success = False
                        break;
                if is_sale_out_success == True:
                    # log.info('all stop price %s'%stop_price, 'position info has been saled out')
                    ready_for_delete[stop_price] = True
        for x in ready_for_delete:
            if ready_for_delete[x] == True:
                del g.position[x]
                log.info('stop price:%s'%x, 'position_info delete')
    log.info('sale out finished...')


def process_dividend(open_price, out_value):
    if abs(g.last_open-open_price) >= g.last_open/2-g.n_value and g.position:
        log.info('dividend happend, sale out all positions firstly! last open:%s'%g.last_open, 'now open:%s'%open_price)
        oto = order_target_value(g.security, 0, LimitOrderStyle(out_value))
        if oto is not None:
            # log.info('dividend happend, sale out all position unit as price :%s'%oto.price, 'want price:%s'%out_value,'successfully' )
            g.position.clear()
            g.last_buyin = 0
    
# Turtle Trade Method
def turtle_trade_method(context):
    process_n_value(context)
    if g.n_value == 0:
        return
    bt_data = calculate_market_in_and_out_price_with_long(context, 20, 10)
    stock_data = get_price(g.security, end_date=context.current_dt.date(), frequency='daily', fields=['high', 'open', 'low'], count=1)
    open_value = stock_data['open'][0]
    high_value = stock_data['high'][0]
    low_value = stock_data['low'][0]
    log.info("open_value:%s"%open_value, "high_value:%s"%high_value, 'low_value:%s'%low_value, 'in_value:%s'%bt_data['in_val'], 'out_value:%s'%bt_data['out_val'])
    
    process_dividend(open_value, bt_data['out_val'])
    sale_out_market(context, bt_data, low_value)
    buy_in_market(context, bt_data, open_value, high_value)
    g.last_open = open_value
#### ###################################################################################


# 开盘前运行函数
def before_market_open(context):
    # 输出运行时间
    log.info('函数运行时间(before_market_open)：'+str(context.current_dt.time()))
    g.bstock_start = context.current_dt.date() >= g.stock_start_date
    # 给微信发送消息（添加模拟交易，并绑定微信生效）
    # send_message('美好的一天~')

    # 要操作的股票：平安银行（g.为全局变量）


## 开盘时运行函数
def market_open(context):
    if g.bstock_start is True:
        log.info('函数运行时间(market_open):'+str(context.current_dt.time()))
    
        # five_days_mean(context)
        turtle_trade_method(context)


## 收盘后运行函数
def after_market_close(context):
    if g.bstock_start is True:
        log.info(str('函数运行时间(after_market_close):'+str(context.current_dt.time())))
        #得到当天所有成交记录
        trades = get_trades()
        for _trade in trades.values():
            log.info('成交记录：'+str(_trade))
            for _orderid in g.buyin_order_not_finished:
                if _trade.order_id == _orderid:
                    # log.info('find not finished buyin order, id:%s'%_orderid)
                    if g.stop_price != 0:
                        # log.info('find stop price:%s'%g.stop_price)
                        g.position[g.stop_price].append(position_info(buyin=_trade.price, count=_trade.amount, jumpflag=g.jump_flag))
                    else:
                        new_stop_price = _trade.price-2*g.n_value
                        g.jump_flag += 1 
                        # log.info('not find stop price, use new stop price%s'%new_stop_price,'instead')
                        g.position[new_stop_price] = [position_info(buyin=_trade.price, count=_trade.amount, jumpflag=g.jump_flag)]
                        g.last_buyin = _trade.price
        g.buyin_order_not_finished = []
        g.stop_price = 0
        for key in list(g.position.keys()):
            for pi in g.position[key]:
                log.info('now position info, stop price:%s'%key, 'buyin :%s'%pi.buyin, 'count:%s'%pi.count, 'jumpflag: %s'%pi.jumpflag)
                    
        log.info('一天结束')
        log.info('##############################################################')
